In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample data
eng_sentences = ['hello', 'how are you', 'thank you']
fra_sentences = ['bonjour', 'comment ça va', 'merci']

# Tokenization
eng_tokenizer = Tokenizer()
fra_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(eng_sentences)
fra_tokenizer.fit_on_texts(fra_sentences)

eng_seqs = eng_tokenizer.texts_to_sequences(eng_sentences)
fra_seqs = fra_tokenizer.texts_to_sequences(fra_sentences)

eng_padded = pad_sequences(eng_seqs, padding='post')
fra_padded = pad_sequences(fra_seqs, padding='post')

# Model parameters
embedding_dim = 64
units = 128
input_vocab_size = len(eng_tokenizer.word_index) + 1
target_vocab_size = len(fra_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Prepare decoder target data
decoder_target_data = np.expand_dims(fra_padded, -1)

# Train model
model.fit([eng_padded, fra_padded], decoder_target_data, batch_size=2, epochs=500, verbose=0)

# Simple prediction
test_input = pad_sequences(eng_tokenizer.texts_to_sequences(['thank you']), maxlen=3, padding='post')
decoder_input = np.zeros((1, 3))  # assume empty start for simplicity
preds = model.predict([test_input, decoder_input])
predicted_ids = np.argmax(preds[0], axis=-1)

# Convert prediction to text
reverse_fra_word_index = {i: word for word, i in fra_tokenizer.word_index.items()}
translated_sentence = ' '.join([reverse_fra_word_index.get(i, '') for i in predicted_ids])
print("Translated Sentence:", translated_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
Translated Sentence: merci  
